In [ ]:
!pip install xgboost

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from scipy.stats import zscore
from scipy import stats
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv(r'C:\Users\mehak\Documents\GitHub\Machine-Learning-project\Datasets\Video_Games_Sales_as_at_22_Dec_2016.csv')
print(df.head())

                       Name Platform  Year_of_Release         Genre Publisher  \
0                Wii Sports      Wii           2006.0        Sports  Nintendo   
1         Super Mario Bros.      NES           1985.0      Platform  Nintendo   
2            Mario Kart Wii      Wii           2008.0        Racing  Nintendo   
3         Wii Sports Resort      Wii           2009.0        Sports  Nintendo   
4  Pokemon Red/Pokemon Blue       GB           1996.0  Role-Playing  Nintendo   

   NA_Sales  EU_Sales  JP_Sales  Other_Sales  Global_Sales  Critic_Score  \
0     41.36     28.96      3.77         8.45         82.53          76.0   
1     29.08      3.58      6.81         0.77         40.24           NaN   
2     15.68     12.76      3.79         3.29         35.52          82.0   
3     15.61     10.93      3.28         2.95         32.77          80.0   
4     11.27      8.89     10.22         1.00         31.37           NaN   

   Critic_Count User_Score  User_Count Developer Rating 

In [4]:
# Handling Missing Values
print("Missing Values:\n", df.isnull().sum())

Missing Values:
 Name                  2
Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
Critic_Score       8582
Critic_Count       8582
User_Score         6704
User_Count         9129
Developer          6623
Rating             6769
dtype: int64


In [5]:
df.dropna(inplace=True)
df

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,65.0,8.5,431.0,Nintendo,E
7,Wii Play,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,41.0,6.6,129.0,Nintendo,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16667,E.T. The Extra-Terrestrial,GBA,2001.0,Action,NewKidCo,0.01,0.00,0.00,0.00,0.01,46.0,4.0,2.4,21.0,Fluid Studios,E
16677,Mortal Kombat: Deadly Alliance,GBA,2002.0,Fighting,Midway Games,0.01,0.00,0.00,0.00,0.01,81.0,12.0,8.8,9.0,Criterion Games,M
16696,Metal Gear Solid V: Ground Zeroes,PC,2014.0,Action,Konami Digital Entertainment,0.00,0.01,0.00,0.00,0.01,80.0,20.0,7.6,412.0,Kojima Productions,M
16700,Breach,PC,2011.0,Shooter,Destineer,0.01,0.00,0.00,0.00,0.01,61.0,12.0,5.8,43.0,Atomic Games,T


In [6]:
# Replace 'tbd' with NaN
df['User_Score'] = df['User_Score'].replace('tbd', np.nan)
# Convert to float
df['User_Score'] = pd.to_numeric(df['User_Score'])

In [7]:
df = df.dropna(subset=['User_Score'])

In [8]:
categorical_cols = df.select_dtypes(include=['object']).columns
print (categorical_cols)

Index(['Name', 'Platform', 'Genre', 'Publisher', 'Developer', 'Rating'], dtype='object')


In [9]:
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Use only these 7 features
features_to_use = [
    'Platform', 'Year_of_Release', 'Publisher',
    'Critic_Score', 'User_Score', 'Rating', 'Developer'
]


# Label encode categorical features
categorical_cols = ['Platform', 'Publisher', 'Rating', 'Developer','Genre']
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

X = df[features_to_use].copy()
y = df['Genre']  # Target

# Save the selected features list
selected_features = features_to_use

# Scale numerical features
scaler = StandardScaler()
X_scaled = X.copy()
X_scaled[['Year_of_Release', 'Critic_Score', 'User_Score']] = scaler.fit_transform(
    X[['Year_of_Release', 'Critic_Score', 'User_Score']]
)

In [10]:
X

,Platform,Year_of_Release,Publisher,Critic_Score,User_Score,Rating,Developer
0,12,2006.0,160,76.0,8.0,1,771
2,12,2008.0,160,82.0,8.3,1,771
3,12,2009.0,160,80.0,8.0,1,771
6,2,2006.0,160,89.0,8.5,1,771
7,12,2006.0,160,58.0,6.6,1,771
...,...,...,...,...,...,...,...
16667,3,2001.0,158,46.0,2.4,1,411
16677,3,2002.0,146,81.0,8.8,4,260
16696,5,2014.0,123,80.0,7.6,4,617
16700,5,2011.0,53,61.0,5.8,6,99


In [11]:
y

0        10
2         6
3        10
6         4
7         3
         ..
16667     0
16677     2
16696     0
16700     8
16706    11
Name: Genre, Length: 6825, dtype: int64

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [14]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "XGBoost": XGBClassifier(eval_metric='mlogloss')
}

In [15]:
# Train and evaluate
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    print(f"{name} Accuracy: {acc:.4f}")

c:\Users\mehak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 0.3333
Decision Tree Accuracy: 0.5187
Random Forest Accuracy: 0.5634
Gradient Boosting Accuracy: 0.5399
XGBoost Accuracy: 0.6015


In [16]:
import joblib

joblib.dump(models['XGBoost'], 'video_game_sales_model.joblib')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(label_encoders, 'label_encoders.pkl')
joblib.dump(selected_features, 'feature_info.pkl')


['feature_info.pkl']

In [17]:
import joblib
import numpy as np
import pandas as pd

# Load the saved artifacts
model = joblib.load('video_game_sales_model.joblib')
scaler = joblib.load('scaler.pkl')
label_encoders = joblib.load('label_encoders.pkl')
selected_features = joblib.load('feature_info.pkl')

def get_user_input():
    print("Enter the following feature values:")

    # Collect inputs as strings
    user_data = {}

    # For categorical columns, show label encoder classes so user can enter known values
    for col in ['Platform', 'Publisher', 'Rating', 'Developer']:
        le = label_encoders[col]
        print(f"\nAvailable {col} options:")
        print(list(le.classes_))
        val = input(f"Enter {col}: ")
        # Convert to encoded int, or -1 if not found (handle gracefully)
        if val in le.classes_:
            user_data[col] = le.transform([val])[0]
        else:
            print(f"Warning: '{val}' not found in known {col} classes. Encoding as -1.")
            user_data[col] = -1  # or handle unknown better if needed

    # Numerical features
    # Year_of_Release (int)
    while True:
        try:
            user_data['Year_of_Release'] = int(input("\nEnter Year_of_Release (e.g., 2015): "))
            break
        except ValueError:
            print("Invalid input. Please enter an integer year.")

    # Critic_Score (float)
    while True:
        try:
            user_data['Critic_Score'] = float(input("Enter Critic_Score (0-100): "))
            break
        except ValueError:
            print("Invalid input. Please enter a number.")

    # User_Score (float)
    while True:
        try:
            user_data['User_Score'] = float(input("Enter User_Score (0-10): "))
            break
        except ValueError:
            print("Invalid input. Please enter a number.")

    return user_data

def preprocess_and_predict(user_data):
    # Convert user data dict to DataFrame
    df = pd.DataFrame([user_data])

    # Scale numerical columns
    num_cols = ['Year_of_Release', 'Critic_Score', 'User_Score']
    df[num_cols] = scaler.transform(df[num_cols])

    # Ensure columns are in correct order
    X = df[selected_features]

    # Predict
    pred_encoded = model.predict(X)[0]

    # Decode predicted genre label
    genre_le = label_encoders['Genre']
    pred_genre = genre_le.inverse_transform([pred_encoded])[0]

    return pred_genre

if __name__ == "__main__":
    user_input = get_user_input()
    prediction = preprocess_and_predict(user_input)
    print(f"\nPredicted Game Genre: {prediction}")


Enter the following feature values:

Available Platform options:
['3DS', 'DC', 'DS', 'GBA', 'GC', 'PC', 'PS', 'PS2', 'PS3', 'PS4', 'PSP', 'PSV', 'Wii', 'WiiU', 'X360', 'XB', 'XOne']

Available Publisher options:
['10TACLE Studios', '1C Company', '2D Boy', '3DO', '505 Games', '5pb', 'AQ Interactive', 'ASCII Entertainment', 'Acclaim Entertainment', 'Ackkstudios', 'Acquire', 'Activision', 'Activision Blizzard', 'Activision Value', 'Agatsuma Entertainment', 'Agetec', 'Aksys Games', 'Alternative Software', 'Aqua Plus', 'Arc System Works', 'Ascaron Entertainment GmbH', 'Aspyr', 'Atari', 'Atlus', 'Avalon Interactive', 'Avanquest', 'BAM! Entertainment', 'Banpresto', 'Bethesda Softworks', 'Bigben Interactive', 'Black Bean Games', 'Black Label Games', 'Blue Byte', 'Bohemia Interactive', 'Brash Entertainment', 'CCP', 'CDV Software Entertainment', 'Capcom', 'Cave', 'City Interactive', 'Cloud Imperium Games Corporation', 'Codemasters', 'Codemasters Online', 'Compile Heart', 'Conspiracy Entertainmen

In [ ]:
!pip install flask pyngrok

In [18]:

import os

# Create templates directory
templates_dir = 'templates'
os.makedirs(templates_dir, exist_ok=True)

# Generate 30 inputs dynamically in the form
index_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <title>Video Game Genre Predictor</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 20px; padding: 20px;
            background-color: #f5f5f5;
        }
        .container {
            max-width: 800px;
            margin: 0 auto;
            padding: 30px;
            background-color: white;
            border-radius: 10px;
            box-shadow: 0 0 20px rgba(0,0,0,0.1);
        }
        h1 {
            color: #2c3e50;
            text-align: center;
            margin-bottom: 30px;
        }
        .form-group {
            margin-bottom: 20px;
        }
        label {
            display: block;
            margin-bottom: 8px;
            font-weight: bold;
            color: #34495e;
        }
        input, select, button {
            width: 100%;
            padding: 12px;
            border: 1px solid #ddd;
            border-radius: 5px;
            font-size: 16px;
            box-sizing: border-box;
        }
        button {
            background-color: #3498db;
            color: white;
            border: none;
            cursor: pointer;
            font-weight: bold;
            margin-top: 20px;
            transition: background-color 0.3s;
        }
        button:hover {
            background-color: #2980b9;
        }
        #result {
            margin-top: 30px;
            padding: 20px;
            background-color: #f8f9fa;
            border-radius: 5px;
            display: none;
        }
        #result h2 {
            color: #2c3e50;
            margin-top: 0;
        }
        #predicted_genre {
            font-size: 18px;
            font-weight: bold;
            color: #27ae60;
        }
        .form-row {
            display: flex;
            gap: 20px;
        }
        .form-row .form-group {
            flex: 1;
        }
    </style>
</head>
<body>
<div class="container">
    <h1>Video Game Genre Predictor</h1>
    <form id="predictForm" onsubmit="event.preventDefault(); predictGenre();">

        <div class="form-row">
            <div class="form-group">
                <label for="platform">Platform:</label>
                <select id="platform" name="platform" required>
                    <option value="">Select Platform</option>
                    <option value="3DS">3DS</option>
                    <option value="DC">DC</option>
                    <option value="DS">DS</option>
                    <option value="GBA">GBA</option>
                    <option value="GC">GC</option>
                    <option value="PC">PC</option>
                    <option value="PS">PS</option>
                    <option value="PS2">PS2</option>
                    <option value="PS3">PS3</option>
                    <option value="PS4">PS4</option>
                    <option value="PSP">PSP</option>
                    <option value="PSV">PSV</option>
                    <option value="Wii">Wii</option>
                    <option value="WiiU">WiiU</option>
                    <option value="X360">X360</option>
                    <option value="XB">XB</option>
                    <option value="XOne">XOne</option>
                </select>
            </div>
            <div class="form-group">
                <label for="year">Year of Release:</label>
                <input type="number" id="year" name="year" min="1980" max="2016" required />
            </div>
        </div>

        <div class="form-group">
            <label for="publisher">Publisher:</label>
            <input type="text" id="publisher" name="publisher" required />
        </div>

        <div class="form-group">
            <label for="developer">Developer:</label>
            <input type="text" id="developer" name="developer" required />
        </div>

        <div class="form-row">
            <div class="form-group">
                <label for="critic_score">Critic Score (0-100):</label>
                <input type="number" id="critic_score" name="critic_score" min="0" max="100" step="1" required />
            </div>
            <div class="form-group">
                <label for="user_score">User Score (0-10):</label>
                <input type="number" id="user_score" name="user_score" min="0" max="10" step="0.1" required />
            </div>
        </div>

        <div class="form-group">
            <label for="rating">Rating:</label>
            <select id="rating" name="rating" required>
                <option value="">Select Rating</option>
                <option value="AO">AO</option>
                <option value="E">E</option>
                <option value="E10+">E10+</option>
                <option value="K-A">K-A</option>
                <option value="M">M</option>
                <option value="RP">RP</option>
                <option value="T">T</option>
            </select>
        </div>

        <button type="submit">Predict Genre</button>
    </form>

    <div id="result">
        <h2>Prediction Result</h2>
        <p>Predicted Genre: <span id="predicted_genre"></span></p>
    </div>
</div>

<script>
async function predictGenre() {
    const form = document.getElementById('predictForm');
    const formData = new FormData(form);

    const jsonData = {};
    formData.forEach((value, key) => {
        jsonData[key] = value;
    });

    try {
        const response = await fetch('/predict', {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify(jsonData)
        });

        const result = await response.json();

        if (result.error) {
            document.getElementById('predicted_genre').innerText = 'Error: ' + result.error;
        } else {
            document.getElementById('predicted_genre').innerText = result.genre;
        }

        document.getElementById('result').style.display = 'block';
    } catch (error) {
        document.getElementById('predicted_genre').innerText = 'Error: ' + error.message;
        document.getElementById('result').style.display = 'block';
    }
}
</script>
</body>
</html>

"""

# Write HTML to file
with open(os.path.join(templates_dir, 'index.html'), 'w') as f:
    f.write(index_html)

In [ ]:
import os
from flask import Flask, request, jsonify, render_template
import joblib
import pandas as pd
from pyngrok import ngrok


app = Flask(__name__)



# Load saved objects
model = joblib.load('video_game_sales_model.joblib')
scaler = joblib.load('scaler.pkl')
label_encoders = joblib.load('label_encoders.pkl')
selected_features = joblib.load('feature_info.pkl')


def encode_or_minus_one(le, val):
    if val and val in le.classes_:
        return le.transform([val])[0]
    else:
        return -1  # encode unknown or missing values as -1


@app.route('/')
def home():
    return render_template('index.html')


@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()

        platform_raw = data.get('platform')
        rating_raw = data.get('rating')
        publisher_raw = data.get('publisher')
        developer_raw = data.get('developer')

        platform_enc = encode_or_minus_one(label_encoders['Platform'], platform_raw)
        rating_enc = encode_or_minus_one(label_encoders['Rating'], rating_raw)
        publisher_enc = encode_or_minus_one(label_encoders['Publisher'], publisher_raw)
        developer_enc = encode_or_minus_one(label_encoders['Developer'], developer_raw)

        input_dict = {
            'Platform': platform_enc,
            'Year_of_Release': int(data.get('year')),
            'Publisher': publisher_enc,
            'Critic_Score': float(data.get('critic_score')),
            'User_Score': float(data.get('user_score')),
            'Rating': rating_enc,
            'Developer': developer_enc
        }

        input_df = pd.DataFrame([input_dict])

        # Scale numeric columns
        num_cols = ['Year_of_Release', 'Critic_Score', 'User_Score']
        input_df[num_cols] = scaler.transform(input_df[num_cols])

        # Select features in the right order
        input_df = input_df[selected_features]

        pred_encoded = model.predict(input_df)[0]

        pred_genre = label_encoders['Genre'].inverse_transform([pred_encoded])[0]

        return jsonify({'genre': pred_genre})

    except Exception as e:
        return jsonify({'error': str(e)})


if __name__ == '__main__':
    port = 5001
    ngrok.set_auth_token("2xYZF9UE3YezFSf5tD3ljuLUEPZ_MWhHUxK2tDiKQFsa3UX9")
    public_url = ngrok.connect(port)
    print(f"Ngrok Tunnel URL: {public_url}")

    app.run(port=5001)


Ngrok Tunnel URL: NgrokTunnel: "https://52fc-2402-ad80-aa-1c2f-e4fa-3626-ee2d-3c91.ngrok-free.app" -> "http://localhost:5001"
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [26/May/2025 22:55:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2025 22:55:44] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [26/May/2025 22:56:11] "POST /predict HTTP/1.1" 200 -
